In [ ]:
!nvidia-smi

Thu Nov 24 03:18:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Environment

In [ ]:
%%capture

import math
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

!pip install transformers
!pip install sentencepiece
# ! pip install TorchCRF
from transformers import AutoTokenizer, AutoModel, RobertaTokenizer
# from TorchCRF import CRF
tokenizer = AutoTokenizer.from_pretrained("unitary/toxic-bert", add_prefix_space=True)
from tqdm.notebook import tqdm
from sklearn.metrics import classification_report
import pandas as pd

In [ ]:
import numpy as np

# Load

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import pickle
path='/content/drive/MyDrive/CONDA/data/'
file_name=('train_new.pkl', 'valid_new.pkl', 'test_new.pkl')
with open(file=path+file_name[0], mode='rb') as f:
    train_data=pickle.load(f)
with open(file=path+file_name[1], mode='rb') as f:
    valid_data=pickle.load(f)
with open(file=path+file_name[2], mode='rb') as f:
    test_data=pickle.load(f)

In [ ]:
training_data = pd.read_csv("/content/drive/MyDrive/NER/CONDA_train_leaderboard.csv")
testing_data = pd.read_csv("/content/drive/MyDrive/NER/CONDA_valid_leaderboard.csv")

training_post=training_data['tokenized'].tolist()
training_labels=training_data['slotClasses'].tolist()
training_int = training_data['intentClass'].tolist()

testing_post=testing_data['tokenized'].tolist()
testing_labels=testing_data['slotClasses'].tolist()
testing_int = testing_data['intentClass'].tolist()

truelabels = pd.read_csv("/content/drive/MyDrive/NER/CONDA_test_leaderboard.csv")
truelabels = truelabels['tokenized'].tolist()

In [ ]:
def remove_punctuation(data):
    processed_data = []
    for string in data:
        string = string.replace('[', '')
        string = string.replace(']', '')
        string = string.replace('!', '')
        string = string.replace('#', '')
        processed_data.append(string.strip().lower())
    return processed_data
training_post = remove_punctuation(training_post)
testing_post = remove_punctuation(testing_post)
truelabels = remove_punctuation(truelabels)


def tokenization(sentences):
    ls = []
    for sent in sentences:
        sent = sent.split()
        ls.append(sent)
    return ls

training_post = tokenization(training_post)
testing_post = tokenization(testing_post)
truelabels = tokenization(truelabels)

In [ ]:
training_post

In [ ]:
train_data

,token_white_space,intent_label,intent_encode,slot_label,slot_encode
0,[wow],O,3,[O],[2]
1,[WTF],O,3,[T],[6]
2,"[wpe, wpe]",O,3,"[O, O]","[2, 2]"
3,[hahaha],O,3,[O],[2]
4,[wtf],O,3,[T],[6]
...,...,...,...,...,...
26081,"[i, bet]",O,3,"[P, O]","[3, 2]"
26082,"[cant, believe, it]",O,3,"[O, O, P]","[2, 2, 3]"
26083,[AH],O,3,[O],[2]
26084,"[dayuuuuuum, SEPA, lool]",O,3,"[O, SEPA, O]","[2, 5, 2]"


In [ ]:
# max_length=0
# for i in range(train_data.shape[0]):
#     inputs= tokenizer.encode_plus(train_data.token_white_space.tolist()[i], is_split_into_words=True)
#     max_length=max(max_length, len(inputs['input_ids']))
# for i in range(valid_data.shape[0]):
#     inputs= tokenizer.encode_plus(valid_data.token_white_space.tolist()[i], is_split_into_words=True)
#     max_length=max(max_length, len(inputs['input_ids']))
# for i in range(test_data.shape[0]):
#     inputs= tokenizer.encode_plus(test_data.token_white_space.tolist()[i], is_split_into_words=True)
#     max_length=max(max_length, len(inputs['input_ids']))
# max_length

In [ ]:
record_intent=list(set(training_int + testing_int))
record_intent.sort()
record_intent

['A', 'E', 'I', 'O']

In [ ]:
from sklearn import preprocessing
label_encoder_intent = preprocessing.LabelEncoder()
label_encoder_intent.fit(record_intent)
print(label_encoder_intent.classes_)
train_intent_encode=label_encoder_intent.transform(training_int).tolist()
valid_intent_encode=label_encoder_intent.transform(testing_int).tolist()

['A' 'E' 'I' 'O']


In [ ]:
list(label_encoder_intent.inverse_transform([2]))

['I']

In [ ]:
train_data = pd.DataFrame(np.array([training_post+testing_post[:-100],training_int+testing_int[:-100],train_intent_encode+valid_intent_encode[:-100]]),
                          index=['token_white_space', 'intent_label', 'intent_encode']).T

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [ ]:
valid_data = pd.DataFrame(np.array([testing_post[-100:],testing_int[-100:],valid_intent_encode[-100:]]),
                          index=['token_white_space', 'intent_label', 'intent_encode']).T

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [ ]:
test_data = pd.DataFrame(np.array([truelabels]),
                          index=['token_white_space']).T

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [ ]:
test_data


,token_white_space
0,[deff]
1,[sup]
2,"[like, i, sad, sepa, buy, back, to, dead]"
3,"[hi, aba]"
4,"[kiss, u]"
...,...
8698,"[they, call, him, 1, vs, 5, od, sepa, fear, him]"
8699,"[i, won, mid, you, daggot]"
8700,"[strong, carries, void, sepa, thanks, for, foc..."
8701,"[abia, void, abia, legion]"


In [ ]:
# train_data.insert(2, 'intent_encode', train_intent_encode)
# train_data.insert(train_data.shape[-1], 'slot_encode', train_slot_encode)

# valid_data.insert(2, 'intent_encode', valid_intent_encode)
# valid_data.insert(valid_data.shape[-1], 'slot_encode', valid_slot_encode)

In [ ]:
valid_data

,token_white_space,intent_label,intent_encode
0,[gg],O,3
1,[gg],O,3
2,"[gg, sepa, nice, late, game]",O,3
3,[fuk],E,1
4,[;)],O,3
...,...,...,...
8701,"[rofl, sepa, sf, sepa, report, sf, sepa, sf, u...",A,0
8702,[ggwp],O,3
8703,[kappa],O,3
8704,"[good, luck, have, f, uck]",E,1


In [ ]:
train_data.head(3)

,token_white_space,intent_label,intent_encode
0,[wow],O,3
1,[wtf],O,3
2,"[wpe, wpe]",O,3


In [ ]:
valid_data.head(3)

,token_white_space,intent_label,intent_encode,slot_label,slot_encode
0,[GG],O,3,[S],[4]
1,[gg],O,3,[S],[4]
2,"[GG, SEPA, nice, late, game]",O,3,"[S, SEPA, O, O, O]","[4, 5, 2, 2, 2]"


In [ ]:
test_data.head(3)

,token_white_space
0,[deff]
1,[sup]
2,"[like, I, sad, SEPA, buy, back, to, dead]"


# Dataloader

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, df, tokenizer, mode):
        self.df = df
        self.tokenizer = tokenizer
        self.mode = mode
        
        # self.pre_token = df.token_white_space #series
        # self.intent_label = df.intent_encode #series
        # self.slot_label = df.slot_encode #series

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        pre_token =  self.df.token_white_space[index] #list
        inputs = self.tokenizer.encode_plus(
                pre_token,
                truncation=True,
                add_special_tokens=True,
                max_length=136,
                padding='max_length',
                is_split_into_words=True #因为tokens是已经分好的词，特殊情况
                # return_tensors="pt"
            )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        if self.mode != 'test':
            intent_label = self.df.intent_encode[index] #int
            return {
                'ids': torch.tensor(ids, dtype=torch.long),
                'mask': torch.tensor(mask, dtype=torch.long),
                'intent_label': torch.tensor(intent_label, dtype=torch.long),
            }

        else:
            return {
                'ids': torch.tensor(ids, dtype=torch.long),
                'mask': torch.tensor(mask, dtype=torch.long),
            }

In [ ]:
def BuildDataloader(df, tokenizer, mode, batch_size, shuffle):
    cumstom_data = CustomDataset(df, tokenizer, mode)
    data_loader = DataLoader(cumstom_data, batch_size=batch_size, num_workers=2, shuffle=shuffle, pin_memory=True)
    return data_loader

# Model

In [ ]:
class JointModel(nn.Module):
    def __init__(self, intent_num_labels=4):
        super().__init__()
        self.backbone = AutoModel.from_pretrained('unitary/toxic-bert')

        input_dim = self.backbone.config.hidden_size
        intent_out_dim = intent_num_labels

        #Instantiate an intent classifier
        self.intent_classifier = torch.nn.Sequential(
            torch.nn.Linear(input_dim, input_dim),
            torch.nn.GELU(),
            torch.nn.Dropout(0.1),
            torch.nn.Linear(input_dim, intent_num_labels)
        )




    def forward(self, input_ids, attention_mask):
        outputs = self.backbone(input_ids, attention_mask, output_hidden_states=True) #outputs=(last_hidden_state，pooler_output，hidden_states)
        # Extract the last hidden state of the token [CLS] for classification task
        #main1
        # CLS = outputs[0][:, 0, :]

        #main2
        CLS=outputs[1]
        # hidden_states=outputs[2] # tuple
        # sequence_output = outputs[0] # (B,N,H)
        #while outputs.pooler_output is the token of last_hidden_state_cls, processed by a linear layer and tanh activation function
        # hidden_states = outputs[2] #(layers, batch_size, sequence_length, hidden_size)
        # concated_token_output = torch.cat(tuple([hidden_states[i] for i in [-4, -3, -2, -1]]), dim=-1)#(batch_size, sequence_length, 4*hidden_size)
                                    # sum along the axis                             divide by attention_mask along that axis
        # mean_pooled_output  = concated_token_output.sum(axis =1) / attention_mask.sum(axis=-1).unsqueeze(-1) #(batch_size, 4*hidden_size)
        # out = self.dropout(mean_pooled_output)

        intent_logits = self.intent_classifier(CLS) #(batch_size, intent_num_labels)
        return intent_logits

# Train

In [ ]:
def ratio(step, model_size, factor, warmup):
    if step == 0:
        step = 1
    return factor * (model_size ** (-0.5) * min(step ** (-0.5), step * warmup ** (-1.5)))

def train(model, optimizer, lr_scheduler, criterion, training_loader): 
    model.train()
    #用于累计一个epoch的loss
    intent_loss_recorder = []


    #用于累计一个epoch的真实结果和预测结果
    intent_target_recorder = []
    intent_predition_recorder = []


    for data in tqdm(training_loader, total=len(training_loader)): #len(training_loader)=data_num/batch_size
        input_ids = data['ids'].to(device)
        attention_mask = data['mask'].to(device)
        intent_target =data['intent_label'].to(device) #shape:(batch_size, )

        optimizer.zero_grad()

        #inference        
        intent_logits = model(input_ids, attention_mask) #logit.shape=(batch_size, num_labels)
        
        #task1
        intent_loss = criterion(intent_logits, intent_target) #signal number 
        intent_loss_recorder.append(intent_loss.item())


        intent_loss.backward()

        optimizer.step()
        
        lr_scheduler.step()
    

        #用于计算intent的F1
        intent_target_recorder.extend(intent_target.cpu().detach().numpy().tolist()) 
        intent_predition_recorder.extend(torch.argmax(intent_logits, -1).cpu().detach().numpy().tolist())




    #计算一个epoch的平均损失，平均体现在batch_size
    intent_loss_epoch = np.average(intent_loss_recorder)

    #计算一个epoch中intent的结果
    intent_name=['A', 'E', 'I', 'O']
    intent_result=classification_report(intent_target_recorder, intent_predition_recorder, zero_division=0,
                                        target_names=intent_name, output_dict=True)
    intent_df=pd.DataFrame(intent_result).T
    intent_df['support'] = intent_df.support.apply(int)
    intent_df=intent_df.style.background_gradient(cmap='rainbow', subset=pd.IndexSlice[:intent_name[-1], :'f1-score'])



    return{
        'intent_loss': intent_loss_epoch,
        'intent_result': intent_df
        }

# Valid

In [ ]:
def validation(model, criterion, validation_loader):
    model.eval()
    #用于累计一个epoch的loss
    intent_loss_recorder=[]


    #用于累计一个epoch的真实结果和预测结果
    intent_target_recorder = []
    intent_predition_recorder = []

    
    with torch.no_grad():
        for data in tqdm(validation_loader, total=len(validation_loader)):
            input_ids = data['ids'].to(device)
            attention_mask = data['mask'].to(device)
            intent_target =data['intent_label'].to(device) #shape:(batch_size, )

            intent_logits = model(input_ids, attention_mask) #logit.shape=(batch_size, num_labels)

            #task1
            intent_loss = criterion(intent_logits, intent_target) #signal number 
            intent_loss_recorder.append(intent_loss.item())

            #用于计算intent的F1
            intent_target_recorder.extend(intent_target.cpu().detach().numpy().tolist()) 
            intent_predition_recorder.extend(torch.argmax(intent_logits, -1).cpu().detach().numpy().tolist())


        #计算一个epoch的平均损失，平均体现在batch_size
        intent_loss_epoch = np.average(intent_loss_recorder)

        #计算一个epoch中intent和slot的结果
        intent_name=['A', 'E', 'I', 'O']
        intent_result=classification_report(intent_target_recorder, intent_predition_recorder, zero_division=0,
                                            target_names=intent_name, output_dict=True)
        intent_df=pd.DataFrame(intent_result).T
        intent_df['support'] = intent_df.support.apply(int)
        intent_df=intent_df.style.background_gradient(cmap='rainbow', subset=pd.IndexSlice[:intent_name[-1], :'f1-score'])

        
    return{
        'intent_loss': intent_loss_epoch,
        'intent_result': intent_df,
        }

# Fit

In [ ]:
def fit(model, epochs, optimizer, lr_scheduler, criterion, training_loader, validation_loader):
    #train相关的数值记录
    train_intent_loss=[]


    #valid相关的数值记录
    valid_intent_loss=[]


    #模型名字
    model_name=model.__class__.__name__

    # early stopping相关初始参数
    best_valid_intent_loss = np.iinfo(np.int32).max
    best_f1 = np.iinfo(np.int32).min
    cumulative_descent = 0
    
    for epoch in range(epochs):
        flag=False

        #inference process
        train_outputs = train(model, optimizer, lr_scheduler, criterion, training_loader)

        #train_loss相关
        train_intent_loss.append(train_outputs['intent_loss'])


        #validation process
        valid_outputs = validation(model, criterion, validation_loader)

        #valid_loss相关
        valid_intent_loss.append(valid_outputs['intent_loss'])

        torch.save(model.state_dict(), f'{epoch+1}_checkpoint_bestloss.pt')
        #Early stopping
        if best_valid_intent_loss > valid_outputs['intent_loss']:
            flag=True
            cumulative_descent = 0
            best_valid_intent_loss =  valid_outputs['intent_loss']
            # torch.save(model.state_dict(), f'{model_name}_checkpoint_bestloss.pt')
        else:
            cumulative_descent += 1

        

        # print(f"Epoch: {(epoch+1):02d}丨Train->    total_loss: {(train_total_loss[-1]):.4f}丨intent_loss: {(train_intent_loss[-1]):.4f}丨intent_f1: {(100*(train_intent_f1[-1])):.2f}%丨slot_loss: {(train_slot_loss[-1]):.4f}丨slot_f1: {(100*(train_slot_f1[-1])):.2f}%")
        # print(f"Epoch: {(epoch+1):02d}丨Valid->    total_loss: {(valid_total_loss[-1]):.4f}丨intent_loss: {(valid_intent_loss[-1]):.4f}丨intent_f1: {(100*(valid_intent_f1[-1])):.2f}%丨slot_loss: {(valid_slot_loss[-1]):.4f}丨slot_f1: {(100*(valid_slot_f1[-1])):.2f}%")
        print(f"Epoch: {(epoch+1):02d}丨Train->    intent_loss: {(train_intent_loss[-1]):.4f}")
        print(f'intent result:')
        display(train_outputs['intent_result'])
        print('\n')
        print(f"Epoch: {(epoch+1):02d}丨Valid->    intent_loss: {(valid_intent_loss[-1]):.4f}")
        print(f'intent result:')
        display(valid_outputs['intent_result'])
        
        print(f'Update checkpoint: {flag}丨cumulative_descent: {cumulative_descent}')
        print('=================================================================================================')
        
        if cumulative_descent == 5: 
            break
            
    # return

# Main 5
full data

In [ ]:
model=JointModel().to(device)
batch_size = 16
train_data_loader=BuildDataloader(train_data, tokenizer, mode='train', batch_size=batch_size, shuffle=True)
valid_data_loader=BuildDataloader(valid_data, tokenizer, mode='valid', batch_size=batch_size, shuffle=True)

# intent_weights=torch.tensor([1, 1, 1, 0.85] ,dtype=torch.float).to(device)
criterion= nn.CrossEntropyLoss()

total_epoch = 25
learning_rate = 3e-3
factor =1
model_size = model.backbone.config.hidden_size
# total_step = len(train_loader_bert) * total_epoch
# warmup_ratio = 0.06
# warmup_steps = math.ceil(total_step * warmup_ratio)
warmup_steps = 1200 #Custom or empirical formula

optimizer = optim.AdamW(model.parameters(), lr=learning_rate, betas=(0.9, 0.98), eps=1e-8)
lr_scheduler = optim.lr_scheduler.LambdaLR(optimizer=optimizer, lr_lambda=lambda step: ratio(step, model_size, factor, warmup_steps)) #Noam

In [ ]:
fit(model, total_epoch, optimizer, lr_scheduler, criterion, train_data_loader, valid_data_loader)

  0%|          | 0/2169 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

Epoch: 01丨Train->    intent_loss: 0.6461
intent result:


,precision,recall,f1-score,support
A,0.689281,0.221737,0.335535,2291
E,0.773790,0.697679,0.733766,4697
I,0.536663,0.223155,0.315231,2263
O,0.846624,0.957706,0.898746,25441
accuracy,0.825983,0.825983,0.825983,0
macro avg,0.711590,0.525069,0.570820,34692
weighted avg,0.806153,0.825983,0.801152,34692




Epoch: 01丨Valid->    intent_loss: 0.2256
intent result:


...



Epoch: 06丨Valid->    intent_loss: 0.1552
intent result:


,precision,recall,f1-score,support
A,0.833333,0.625000,0.714286,8
E,0.909091,0.833333,0.869565,12
I,1.000000,1.000000,1.000000,8
O,0.933333,0.972222,0.952381,72
accuracy,0.930000,0.930000,0.930000,0
macro avg,0.918939,0.857639,0.884058,100
weighted avg,0.927758,0.930000,0.927205,100


Update checkpoint: True丨cumulative_descent: 0


  0%|          | 0/2169 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
best_model=JointModel().to(device)
best_model.load_state_dict(torch.load('/content/3_checkpoint_bestloss.pt'))

test_data_loader=BuildDataloader(test_data, tokenizer, mode='test', batch_size=1, shuffle=False)
results=test(best_model, test_data_loader)

intent_answer=[]
for intent in results['intent_predition']:
    intent_answer.append(list(label_encoder_intent.inverse_transform(intent)))

with open('/content/slot.txt', mode='r') as f:
    slot_answer = f.read()

merge_results=[]
s=slot_answer.split('\n')
for j in range(8703):
    merge_results.append(intent_answer[j][0]+s[j][1:])

with open('qq_3.txt', mode='a') as f:
    for i in merge_results:
        f.write(i)
        f.write('\n')

  0%|          | 0/8703 [00:00<?, ?it/s]

# Main 4
使用26000 + 5000

In [ ]:
model=JointModel().to(device)
batch_size = 16
train_data_loader=BuildDataloader(train_data, tokenizer, mode='train', batch_size=batch_size, shuffle=True)
valid_data_loader=BuildDataloader(valid_data, tokenizer, mode='valid', batch_size=batch_size, shuffle=True)

# intent_weights=torch.tensor([1, 1, 1, 0.85] ,dtype=torch.float).to(device)
criterion= nn.CrossEntropyLoss()

total_epoch = 25
learning_rate = 3e-3
factor =1
model_size = model.backbone.config.hidden_size
# total_step = len(train_loader_bert) * total_epoch
# warmup_ratio = 0.06
# warmup_steps = math.ceil(total_step * warmup_ratio)
warmup_steps = 1200 #Custom or empirical formula

optimizer = optim.AdamW(model.parameters(), lr=learning_rate, betas=(0.9, 0.98), eps=1e-8)
lr_scheduler = optim.lr_scheduler.LambdaLR(optimizer=optimizer, lr_lambda=lambda step: ratio(step, model_size, factor, warmup_steps)) #Noam

In [ ]:
fit(model, total_epoch, optimizer, lr_scheduler, criterion, train_data_loader, valid_data_loader)

  0%|          | 0/1943 [00:00<?, ?it/s]

  0%|          | 0/232 [00:00<?, ?it/s]

Epoch: 01丨Train->    intent_loss: 0.6726
intent result:


,precision,recall,f1-score,support
A,0.442408,0.246475,0.316578,2057
E,0.425266,0.591559,0.494815,4194
I,0.914072,0.365174,0.521863,2010
O,0.838991,0.856561,0.847685,22825
accuracy,0.748665,0.748665,0.748665,0
macro avg,0.655184,0.514942,0.545235,31086
weighted avg,0.761785,0.748665,0.743865,31086




Epoch: 01丨Valid->    intent_loss: 0.3565
intent result:


...



Epoch: 08丨Valid->    intent_loss: 0.2759
intent result:


,precision,recall,f1-score,support
A,0.764000,0.789256,0.776423,242
E,0.888224,0.864078,0.875984,515
I,0.914439,0.655172,0.763393,261
O,0.938945,0.966890,0.952713,2688
accuracy,0.919050,0.919050,0.919050,0
macro avg,0.876402,0.818849,0.842128,3706
weighted avg,0.918747,0.919050,0.917205,3706


Update checkpoint: False丨cumulative_descent: 1


  0%|          | 0/1943 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
best_model=JointModel().to(device)
best_model.load_state_dict(torch.load('/content/6_checkpoint_bestloss.pt'))

batch_size = 16
train_data_loader=BuildDataloader(train_data, tokenizer, mode='train', batch_size=batch_size, shuffle=True)
valid_data_loader=BuildDataloader(valid_data, tokenizer, mode='valid', batch_size=batch_size, shuffle=True)

# intent_weights=torch.tensor([1, 1, 1, 0.85] ,dtype=torch.float).to(device)
criterion= nn.CrossEntropyLoss()

total_epoch = 5
learning_rate = 1e-3
factor =1
model_size = best_model.backbone.config.hidden_size
# total_step = len(train_loader_bert) * total_epoch
# warmup_ratio = 0.06
# warmup_steps = math.ceil(total_step * warmup_ratio)
warmup_steps = 1200 #Custom or empirical formula

optimizer = optim.AdamW(best_model.parameters(), lr=learning_rate, betas=(0.9, 0.98), eps=1e-8)
lr_scheduler = optim.lr_scheduler.LambdaLR(optimizer=optimizer, lr_lambda=lambda step: ratio(step, model_size, factor, warmup_steps)) #Noam

In [ ]:
fit(best_model, total_epoch, optimizer, lr_scheduler, criterion, train_data_loader, valid_data_loader)

  0%|          | 0/1943 [00:00<?, ?it/s]

  0%|          | 0/232 [00:00<?, ?it/s]

Epoch: 01丨Train->    intent_loss: 0.2192
intent result:


,precision,recall,f1-score,support
A,0.852652,0.843947,0.848278,2057
E,0.892875,0.872437,0.882537,4194
I,0.893495,0.697015,0.783119,2010
O,0.949239,0.972486,0.960722,22825
accuracy,0.932671,0.932671,0.932671,0
macro avg,0.897065,0.846471,0.868664,31086
weighted avg,0.931639,0.932671,0.931249,31086




Epoch: 01丨Valid->    intent_loss: 0.2789
intent result:


...



Epoch: 03丨Valid->    intent_loss: 0.2797
intent result:


,precision,recall,f1-score,support
A,0.755725,0.818182,0.785714,242
E,0.891129,0.858252,0.874382,515
I,0.869347,0.662835,0.752174,261
O,0.941433,0.962798,0.951996,2688
accuracy,0.917701,0.917701,0.917701,0
macro avg,0.864409,0.825517,0.841066,3706
weighted avg,0.917239,0.917701,0.916279,3706


Update checkpoint: False丨cumulative_descent: 1


  0%|          | 0/1943 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
best_model=JointModel().to(device)
best_model.load_state_dict(torch.load('/content/1_checkpoint_bestloss.pt'))

<All keys matched successfully>

In [ ]:
test_data_loader=BuildDataloader(test_data, tokenizer, mode='test', batch_size=1, shuffle=False)
results=test(best_model, test_data_loader)

  0%|          | 0/8703 [00:00<?, ?it/s]

In [ ]:
intent_answer=[]
for intent in results['intent_predition']:
    intent_answer.append(list(label_encoder_intent.inverse_transform(intent)))

with open('/content/slot.txt', mode='r') as f:
    slot_answer = f.read()

merge_results=[]
s=slot_answer.split('\n')
for j in range(8703):
    merge_results.append(intent_answer[j][0]+s[j][1:])

with open('qq.txt', mode='a') as f:
    for i in merge_results:
        f.write(i)
        f.write('\n')

# Main_3
降低lr继续训练

In [ ]:
best_model=JointModel().to(device)
best_model.load_state_dict(torch.load('/content/drive/MyDrive/NER/JointModel_checkpoint_bestloss_pooling.pt'))

batch_size = 16
train_data_loader=BuildDataloader(train_data, tokenizer, mode='train', batch_size=batch_size, shuffle=True)
valid_data_loader=BuildDataloader(valid_data, tokenizer, mode='valid', batch_size=batch_size, shuffle=True)

# intent_weights=torch.tensor([1, 1, 1, 0.85] ,dtype=torch.float).to(device)
criterion= nn.CrossEntropyLoss()

total_epoch = 25
learning_rate = 1e-3
factor =1
model_size = best_model.backbone.config.hidden_size
# total_step = len(train_loader_bert) * total_epoch
# warmup_ratio = 0.06
# warmup_steps = math.ceil(total_step * warmup_ratio)
warmup_steps = 1200 #Custom or empirical formula

optimizer = optim.AdamW(best_model.parameters(), lr=learning_rate, betas=(0.9, 0.98), eps=1e-8)
lr_scheduler = optim.lr_scheduler.LambdaLR(optimizer=optimizer, lr_lambda=lambda step: ratio(step, model_size, factor, warmup_steps)) #Noam

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
fit(best_model, total_epoch, optimizer, lr_scheduler, criterion, train_data_loader, valid_data_loader)

  0%|          | 0/1631 [00:00<?, ?it/s]

  0%|          | 0/545 [00:00<?, ?it/s]

Epoch: 01丨Train->    intent_loss: 0.2189
intent result:


,precision,recall,f1-score,support
A,0.854471,0.850494,0.852478,1719
E,0.898290,0.878685,0.888379,3528
I,0.893998,0.678297,0.771352,1691
O,0.949392,0.973835,0.961458,19148
accuracy,0.933681,0.933681,0.933681,0
macro avg,0.899038,0.845328,0.868417,26086
weighted avg,0.932635,0.933681,0.932070,26086




Epoch: 01丨Valid->    intent_loss: 0.2812
intent result:


...



Epoch: 05丨Valid->    intent_loss: 0.2845
intent result:


,precision,recall,f1-score,support
A,0.774300,0.810345,0.791912,580
E,0.879654,0.860288,0.869863,1181
I,0.856823,0.660345,0.745862,580
O,0.941973,0.961508,0.951640,6365
accuracy,0.917643,0.917643,0.917643,0
macro avg,0.863187,0.823121,0.839819,8706
weighted avg,0.916676,0.917643,0.916197,8706


Update checkpoint: False丨cumulative_descent: 4


  0%|          | 0/1631 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
torch.save(best_model.state_dict(), f'best_0.8945 .pt')

In [ ]:
test_data_loader=BuildDataloader(test_data, tokenizer, mode='test', batch_size=1, shuffle=False)
results=test(best_model, test_data_loader)

  0%|          | 0/8703 [00:00<?, ?it/s]

In [ ]:
intent_answer=[]
for intent in results['intent_predition']:
    intent_answer.append(list(label_encoder_intent.inverse_transform(intent)))

In [ ]:
out_txt = pd.DataFrame(intent_answer)
out_txt.set_index([0], inplace=True)
out_txt.to_csv('answer.txt',header=False, sep=' ')

In [ ]:
with open('/content/slot.txt', mode='r') as f:
    slot_answer = f.read()

merge_results=[]
s=slot_answer.split('\n')
for j in range(8703):
    merge_results.append(intent_answer[j][0]+s[j][1:])

In [ ]:
with open('qq.txt', mode='a') as f:
    for i in merge_results:
        f.write(i)
        f.write('\n')

# Main_2

还没跑：改total epoch，lr以及用pooler output 

In [ ]:
model=JointModel().to(device)
batch_size = 16
train_data_loader=BuildDataloader(train_data, tokenizer, mode='train', batch_size=batch_size, shuffle=True)
valid_data_loader=BuildDataloader(valid_data, tokenizer, mode='valid', batch_size=batch_size, shuffle=True)

# intent_weights=torch.tensor([1, 1, 1, 0.85] ,dtype=torch.float).to(device)
criterion= nn.CrossEntropyLoss()

total_epoch = 25
learning_rate = 3e-3
factor =1
model_size = model.backbone.config.hidden_size
# total_step = len(train_loader_bert) * total_epoch
# warmup_ratio = 0.06
# warmup_steps = math.ceil(total_step * warmup_ratio)
warmup_steps = 1200 #Custom or empirical formula

optimizer = optim.AdamW(model.parameters(), lr=learning_rate, betas=(0.9, 0.98), eps=1e-8)
lr_scheduler = optim.lr_scheduler.LambdaLR(optimizer=optimizer, lr_lambda=lambda step: ratio(step, model_size, factor, warmup_steps)) #Noam

In [ ]:
model.parameters

In [ ]:
fit(model, total_epoch, optimizer, lr_scheduler, criterion, train_data_loader, valid_data_loader)

  0%|          | 0/1631 [00:00<?, ?it/s]

  0%|          | 0/545 [00:00<?, ?it/s]

Epoch: 01丨Train->    intent_loss: 0.7055
intent result:


,precision,recall,f1-score,support
A,0.148331,0.297266,0.197909,1719
E,0.838935,0.544785,0.660595,3528
I,0.900709,0.300414,0.450554,1691
O,0.829576,0.857217,0.843170,19148
accuracy,0.741969,0.741969,0.741969,0
macro avg,0.679388,0.499920,0.538057,26086
weighted avg,0.790561,0.741969,0.750506,26086




Epoch: 01丨Valid->    intent_loss: 0.3629
intent result:


...

Epoch: 11丨Train->    intent_loss: 0.1865
intent result:


,precision,recall,f1-score,support
A,0.857143,0.879581,0.868217,1719
E,0.907754,0.892574,0.900100,3528
I,0.886938,0.746895,0.810915,1691
O,0.960060,0.974149,0.967053,19148
accuracy,0.942153,0.942153,0.942153,0
macro avg,0.902974,0.873300,0.886571,26086
weighted avg,0.941464,0.942153,0.941363,26086




Epoch: 11丨Valid->    intent_loss: 0.2874
intent result:


,precision,recall,f1-score,support
A,0.764992,0.813793,0.788638,580
E,0.879310,0.863675,0.871422,1181
I,0.851528,0.672414,0.751445,580
O,0.943904,0.959623,0.951698,6365
accuracy,0.917758,0.917758,0.917758,0
macro avg,0.859934,0.827376,0.840801,8706
weighted avg,0.917068,0.917758,0.916604,8706


Update checkpoint: False丨cumulative_descent: 5


# Main_1

In [ ]:
model=JointModel().to(device)
batch_size = 16
train_data_loader=BuildDataloader(train_data, tokenizer, mode='train', batch_size=batch_size, shuffle=True)
valid_data_loader=BuildDataloader(valid_data, tokenizer, mode='valid', batch_size=batch_size, shuffle=True)

# intent_weights=torch.tensor([1, 1, 1, 0.85] ,dtype=torch.float).to(device)
criterion= nn.CrossEntropyLoss()

total_epoch = 20
learning_rate = 1e-3
factor =1
model_size = model.backbone.config.hidden_size
# total_step = len(train_loader_bert) * total_epoch
# warmup_ratio = 0.06
# warmup_steps = math.ceil(total_step * warmup_ratio)
warmup_steps = 1200 #Custom or empirical formula

optimizer = optim.AdamW(model.parameters(), lr=learning_rate, betas=(0.9, 0.98), eps=1e-8)
lr_scheduler = optim.lr_scheduler.LambdaLR(optimizer=optimizer, lr_lambda=lambda step: ratio(step, model_size, factor, warmup_steps)) #Noam

In [ ]:
fit(model, total_epoch, optimizer, lr_scheduler, criterion, train_data_loader, valid_data_loader)

  0%|          | 0/1631 [00:00<?, ?it/s]

  0%|          | 0/545 [00:00<?, ?it/s]

Epoch: 01丨Train->    intent_loss: 0.7979
intent result:


,precision,recall,f1-score,support
A,0.040373,0.007563,0.012739,1719
E,0.825208,0.393424,0.532821,3528
I,0.427027,0.140154,0.211042,1691
O,0.782633,0.961615,0.862941,19148
accuracy,0.768650,0.768650,0.768650,0
macro avg,0.518810,0.375689,0.404886,26086
weighted avg,0.716426,0.768650,0.720009,26086




Epoch: 01丨Valid->    intent_loss: 0.4506
intent result:


...

Epoch: 20丨Train->    intent_loss: 0.2473
intent result:


,precision,recall,f1-score,support
A,0.851432,0.830134,0.840648,1719
E,0.888430,0.853175,0.870445,3528
I,0.926765,0.628622,0.749119,1691
O,0.938214,0.973835,0.955693,19148
accuracy,0.925669,0.925669,0.925669,0
macro avg,0.901210,0.821441,0.853976,26086
weighted avg,0.925020,0.925669,0.923191,26086




Epoch: 20丨Valid->    intent_loss: 0.2893
intent result:


,precision,recall,f1-score,support
A,0.780776,0.798276,0.789429,580
E,0.875540,0.857748,0.866553,1181
I,0.936340,0.608621,0.737722,580
O,0.935401,0.966850,0.950865,6365
accuracy,0.916954,0.916954,0.916954,0
macro avg,0.882014,0.807874,0.836142,8706
weighted avg,0.917042,0.916954,0.914473,8706


Update checkpoint: True丨cumulative_descent: 0


# Test

In [ ]:
def test(model, test_loader):
    model.eval()

    #用于累计一个epoch的预测结果
    intent_predition_recorder = []
    slot_predition_recorder = []
    
    with torch.no_grad():
        for data in tqdm(test_loader, total=len(test_loader)):
            input_ids = data['ids'].to(device)
            attention_mask = data['mask'].to(device)

            intent_logits = model(input_ids, attention_mask) #logit.shape=(batch_size, num_labels)

             #记录intent和slot预测结果
            intent_predition_recorder.append(torch.argmax(intent_logits, -1).cpu().detach().numpy().tolist())

    return{
        'intent_predition': intent_predition_recorder,
        }

In [ ]:
best_model=JointModel().to(device)
best_model.load_state_dict(torch.load('/content/JointModel_checkpoint_bestloss.pt'))

<All keys matched successfully>

In [ ]:
test_data_loader=BuildDataloader(test_data, tokenizer, mode='test', batch_size=1, shuffle=False)

In [ ]:
results=test(best_model, test_data_loader)

  0%|          | 0/8703 [00:00<?, ?it/s]

In [ ]:
intent_answer=[]
for intent in results['intent_predition']:
    intent_answer.append(list(label_encoder_intent.inverse_transform(intent)))

In [ ]:
len(intent_answer)

8703

In [ ]:
out_txt

In [ ]:
out_txt = pd.DataFrame(intent_answer)
out_txt.set_index([0], inplace=True)
out_txt.to_csv('answer.txt',header=False, sep=' ')

# Merge results

In [ ]:
with open('/content/slot.txt', mode='r') as f:
    slot_answer = f.read()

In [ ]:
with open('/content/slot.txt', mode='r') as f:
    slot_answer = f.read()

merge_results=[]
s=slot_answer.split('\n')
for j in range(8703):
    merge_results.append(intent_answer[j][0]+s[j][1:])

In [ ]:
s

In [ ]:
len(merge_results)

8703

In [ ]:
with open('/content/drive/MyDrive/CONDA/data/answer.txt', mode='r') as f:
    sample_answer = f.read()

In [ ]:
sample=sample_answer.split('\n')

In [ ]:
for i in range(8703):
    if len(merge_results[i].split(' '))!=len(sample[i].split(' ')):
        print(i)

In [ ]:
with open('answer.txt', mode='a') as f:
    for i in merge_results:
        f.write(i)
        f.write('\n')

In [ ]:
merge_results